In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL-e kategorii (zaktualizowano o Zupy)
CATEGORY_URLS = {
    "Śniadania": [
        "https://ervegan.com/category/snd/sniadania_na_slodko/",
        "https://ervegan.com/category/snd/sniadania_na_slono/",
    ],
    "Dania główne": [
        "https://ervegan.com/category/dania-glowne/",
    ],
    "Przekąski": [
        "https://ervegan.com/category/przekaski/",
    ],
    "Desery": [
        "https://ervegan.com/category/slodkie/",
    ],
    "Zupy": [
        "https://ervegan.com/category/zupy/",
    ]
}

# Pobieram HTML ze strony
def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Błąd: Nie udało się pobrać strony {url}. Status: {response.status_code}")
        return None

# Pobieram linki do przepisów z danej kategorii
def get_recipe_links(category_url):
    recipe_links = []
    page = 1
    while True:
        page_url = f"{category_url}page/{page}/"
        html_content = fetch_html(page_url)
        if not html_content:
            break

        soup = BeautifulSoup(html_content, "html.parser")
        articles = soup.find_all("article")
        if not articles:
            break

        for article in articles:
            link = article.find("a", href=True)
            if link:
                recipe_links.append(link["href"])
        page += 1

    return recipe_links

# Pobieram szczegóły przepisu z podanego URL
def get_recipe_details(recipe_url, category):
    html_content = fetch_html(recipe_url)
    if not html_content:
        return None

    soup = BeautifulSoup(html_content, "html.parser")
    entry_content = soup.find('div', class_="entry-content")

    title = soup.find('h1', class_="entry-title")
    recipe_title = title.text.strip() if title else "Nieznany przepis"

    # Pobieram URL obrazu
    image_url = None
    if entry_content:
        image_tag = entry_content.find('img')
        if image_tag and image_tag.get('src'):
            image_url = image_tag['src']

    # Pobieram składniki
    ingredients = []
    if entry_content:
        div_tags = entry_content.find_all("div")
        for div in div_tags:
            text = div.get_text(strip=True)
            if len(text.split()) > 2 and any(keyword in text.lower() for keyword in ["szkl", "łyżk", "kostk", "szczypt"]):
                ingredients.append(text)

    # Pobieram instrukcje
    instructions = []
    if entry_content:
        paragraphs = entry_content.find_all("p")
        for paragraph in paragraphs:
            text = paragraph.get_text(strip=True)
            if any(text.startswith(f"{i}.") for i in range(1, 10)):
                instructions.append(text)

    return {
        "title": recipe_title,
        "image_url": image_url,
        "ingredients": ingredients,
        "instructions": instructions,
        "category": category  # Przypisujemy kategorię
    }

# Główna logika scrapowania
def scrape_recipes():
    all_recipes = []

    # Iteracja po wszystkich kategoriach
    for category, urls in CATEGORY_URLS.items():
        for url in urls:
            print(f"Pobieranie przepisów z kategorii: {category} ({url})")
            recipe_links = get_recipe_links(url)

            # Iteracja po linkach do przepisów
            for recipe_link in recipe_links:
                recipe_details = get_recipe_details(recipe_link, category)  # Przekazujemy kategorię
                if recipe_details:
                    all_recipes.append(recipe_details)

    return all_recipes

# Uruchomienie scrapowania
recipes = scrape_recipes()

# Konwersja do DataFrame i zapis do pliku CSV
df = pd.DataFrame(recipes)

# Zapisz dane do CSV
df.to_csv("ervegan_recipes_with_categories.csv", index=False)

print("Dane zostały zapisane do pliku: ervegan_recipes_with_categories.csv")


Pobieranie przepisów z kategorii: Śniadania (https://ervegan.com/category/snd/sniadania_na_slodko/)
Pobieranie przepisów z kategorii: Śniadania (https://ervegan.com/category/snd/sniadania_na_slono/)
Pobieranie przepisów z kategorii: Dania główne (https://ervegan.com/category/dania-glowne/)
Pobieranie przepisów z kategorii: Przekąski (https://ervegan.com/category/przekaski/)
Pobieranie przepisów z kategorii: Desery (https://ervegan.com/category/slodkie/)
Pobieranie przepisów z kategorii: Zupy (https://ervegan.com/category/zupy/)
Błąd: Nie udało się pobrać strony https://ervegan.com/category/zupy/page/3/. Status: 404
Dane zostały zapisane do pliku: ervegan_recipes_with_categories.csv


In [ ]:
import pandas as pd

# Ścieżka do pliku (domyślnie w katalogu głównym po przesłaniu do Colab)
file_path = "/content/ervegan_recipes_with_categories.csv"

# Wczytaj plik CSV do DataFrame
df = pd.read_csv(file_path)

# Wyświetl pierwsze 5 wierszy
df.tail(20)

,title,image_url,ingredients,instructions,category
355,miniTort – czyli wegański biszkopt i kokosowa ...,http://ervegan.com/wp-content/uploads/2014/07/...,['Składnikina średni torcik o średnicy 21 cm:b...,"['1. Mąkę przesiej\xa0do dużej miski, dodaj\xa...",Desery
356,Lodowa tarta z orzechów nerkowca z owocami,http://ervegan.com/wp-content/uploads/2014/07/...,['1. Nerkowce zalej wodą i odstaw\xa0do namocz...,['1. Nerkowce zalej wodą i odstaw\xa0do namocz...,Desery
357,Bezglutenowe racuchy migdałowe z malinami,http://ervegan.com/wp-content/uploads/2014/07/...,['Składniki:160g mąki migdałowej (wysuszonej p...,[],Desery
358,Zupa gochujang,https://ervegan.com/wp-content/uploads/2025/01...,"['Składnikina 4–6 porcji, większy garnek:2 łyż...","['1. Na dnie garnka roztop wegańskie masełko, ...",Zupy
359,Zupa koperkowa,https://ervegan.com/wp-content/uploads/2023/04...,[],[],Zupy
360,Domowa zupka instant,https://ervegan.com/wp-content/uploads/2023/03...,[],"['1. Wszystkie warzywa pokrój, posiekaj lub ob...",Zupy
361,Kimchidorowa,https://ervegan.com/wp-content/uploads/2023/03...,['Składnikina około 4 porcje:1 małacebula1 łyż...,"['1. Cebulę i czosnek posiekaj, podsmaż na ole...",Zupy
362,Zupa instant – misto i tofu,https://ervegan.com/wp-content/uploads/2022/10...,['Składnikina 1 porcję:400mlwody3 łyżkisosusoj...,['1.\xa0Przygotuj wszystkie składniki: posieka...,Zupy
363,Zielony barszcz ukraiński,http://ervegan.com/wp-content/uploads/2022/03/...,"['Składnikina około 6 porcji, garnek 3l:1 więk...",['1. Posiekaną cebulę podsmaż w dużym garnku n...,Zupy
364,Rozgrzewająca zupa cebulowa i nowy Jamie Oliver,http://ervegan.com/wp-content/uploads/2019/10/...,['Składniki na około 5 porcji:5 większych biał...,"['1. Cebulę pokrój w pióra, czosnek w małą kos...",Zupy


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         375 non-null    object
 1   image_url     375 non-null    object
 2   ingredients   375 non-null    object
 3   instructions  375 non-null    object
 4   category      375 non-null    object
dtypes: object(5)
memory usage: 14.8+ KB


In [ ]:
df.describe()

,title,image_url,ingredients,instructions,category
count,375,375,375,375,375
unique,334,334,253,198,5
top,Świąteczny smoothie z kruszonką,http://ervegan.com/wp-content/uploads/2019/12/...,[],[],Dania główne
freq,3,3,94,156,115


In [ ]:
from google.colab import files
files.download('ervegan_recipes_with_categories.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>